In [171]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [172]:
df = pd.read_csv("../data/train.csv")

In [173]:
len(df)

440

In [174]:
count = df[df["stroke"]==0]

In [175]:
len(count)

242

In [176]:
df = df.drop(["Unnamed: 0"],axis=1)
df = df.set_index("id")
label = df["stroke"]
df = df.drop(["stroke"],axis=1)

In [177]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,,
9637,Male,26.0,0,0,Yes,Private,Rural,120.31,22.3,smokes
32257,Female,47.0,0,0,Yes,Private,Urban,210.95,50.1,Unknown
28526,Male,69.0,0,0,Yes,Self-employed,Rural,203.04,33.6,never smoked
56716,Female,26.0,0,0,No,Private,Urban,82.59,29.4,never smoked
72214,Male,61.0,0,0,Yes,Self-employed,Urban,69.15,27.7,formerly smoked


In [178]:
df.isnull().sum()

gender                0
age                   0
hypertension          0
heart_disease         0
ever_married          0
work_type             0
Residence_type        0
avg_glucose_level     0
bmi                  37
smoking_status        0
dtype: int64

In [179]:
df["bmi"].mode()

0    27.3
1    31.0
2    31.4
Name: bmi, dtype: float64

In [180]:
# Find mode value 
mode_1 = df["bmi"].mode()[0]
joblib.dump(mode_1,"../model_clf/mode")


['../model_clf/mode']

In [181]:
mode_1 = df["bmi"].mode()[1]
joblib.dump(mode_1,"../model_tree/mode")

['../model_tree/mode']

In [182]:
def missing_values(df):
    
    mode = joblib.load("../model_clf/mode")
    df = df.fillna(mode)
    return df
    
    

In [183]:
def fit_one_hot(df):
    encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
    encoder.fit(df[["gender","ever_married","work_type","Residence_type"]])
    joblib.dump(encoder,"../model_clf/encoder")

In [184]:
def transform_one_hot(df):
    encoder = joblib.load("../model_clf/encoder")
    result = encoder.transform(df[["gender","ever_married","work_type","Residence_type"]])
    df = df.drop(["gender","work_type","Residence_type","ever_married"],axis=1)
    df[encoder.get_feature_names_out()] = result
    return df

In [185]:
def fit_ordinal(df):
    encoder = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=4)
    encoder.fit(df[["smoking_status"]])
    joblib.dump(encoder,"../model_clf/encoder_ordinal")

In [186]:
def transform_ordinal(df):
    encoder = joblib.load("../model_clf/encoder_ordinal")
    result = encoder.transform(df[["smoking_status"]])
    df[encoder.get_feature_names_out()] = result
    return df

In [187]:
def min_max_scaler_fit(df):
    scaler = MinMaxScaler()
    scaler.fit(df)
    joblib.dump(scaler,"../model_clf/scaler")

In [188]:
def min_max_scaler_transform(df):
    scaler = joblib.load("../model_clf/scaler")
    result = scaler.transform(df)
    df.iloc[:,:] = result
    return df

In [189]:
def train_model(df, label):
    x_train, x_test, y_train, y_test = split_test_train(df,label)
    clf = LogisticRegression()
    clf.fit(x_train,y_train)
    joblib.dump(clf,"../model_clf/clf")
    return clf.score(x_train,y_train), x_train,x_test
    

In [190]:
def split_test_train(df,label):
    X = df
    Y = label
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    return x_train, x_test, y_train, y_test
    

In [191]:
def pipeline(df,label):
    df = missing_values(df)
    
    fit_one_hot(df)
    df = transform_one_hot(df)
    
    fit_ordinal(df)
    df = transform_ordinal(df)
    
    min_max_scaler_fit(df)
    df = min_max_scaler_transform(df)
    score, x_train,x_test = train_model(df, label)
    return score, x_train,x_test
    
    
    